In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# Veriyi yükle
df = pd.read_excel("kart_harcamalar_v2.xlsx")
df["trxdate"] = pd.to_datetime(df["trxdate"])
# trxdate'den ay bilgisini çıkar
df["trx_month"] = df["trxdate"].dt.to_period('M')

# İnceleme periyodu
start_period = pd.Period("2023-07")
end_period = pd.Period("2024-02")

# İnceleme periyoduna göre veriyi filtrele
df_selected_period = df[(df["trx_month"] >= start_period) & (df["trx_month"] <= end_period)]

df_selected_period["TotalAmountLC"] = pd.to_numeric(df_selected_period["TotalAmountLC"], errors='coerce')

# Her bir müşterinin 8 aylık toplam TotalAmountLC'sini hesapla
eight_months_TotalAmountLC = df_selected_period.groupby(["Customer_id"])["TotalAmountLC"].sum().reset_index()

# TotalAmountLC'ye göre sırala
sorted_eight_months_TotalAmountLC = eight_months_TotalAmountLC["TotalAmountLC"].sort_values()

# Müşteri sayısını ve TotalAmountLC'yi al
customers = range(1, len(sorted_eight_months_TotalAmountLC) + 1)
eigtht_months_TotalAmountLC_values = sorted_eight_months_TotalAmountLC.values

total_TotalAmountLC = eight_months_TotalAmountLC["TotalAmountLC"].sum()


# Yüzde 90'lık dilimi bul
percentile_80 = sorted_eight_months_TotalAmountLC.quantile(0.8)

#print("Müşterilerin %80'inin 8 aylık toplam TotalAmountLC'si {:.2f} TL'den küçüktür.(datanın %10 'uyla çalıştığımız için küçük rakam çıkıyor)".format(percentile_80))
#print("Toplam TotalAmountLC:", total_TotalAmountLC)


In [ ]:
#print("Her bir müşteri için 8 aylık toplam TotalAmountLC değerleri:")
#print(eight_months_TotalAmountLC.to_string(index=False))

In [2]:

# Her bir müşterinin her aydaki toplam TotalAmountLC'sini hesapla
monthly_TotalAmountLC_per_customer = df_selected_period.groupby(["Customer_id", "trx_month"])["TotalAmountLC"].sum().reset_index()

# Tüm müşteriler ve tüm aylar için eksik kayıtları doldur
all_customers = df_selected_period["Customer_id"].unique()
all_months = pd.period_range(start=start_period, end=end_period, freq='M')

# Tüm müşteri ve ay kombinasyonlarını içeren bir DataFrame oluştur
customer_month_combinations = pd.MultiIndex.from_product([all_customers, all_months], names=['Customer_id', 'trx_month'])
full_customer_month_TotalAmountLC = pd.DataFrame(index=customer_month_combinations).reset_index()

# Monthly TotalAmountLC ile tam veri çerçevesini birleştirme
full_customer_month_TotalAmountLC = pd.merge(full_customer_month_TotalAmountLC, monthly_TotalAmountLC_per_customer, on=["Customer_id", "trx_month"], how="left")

# Eksik TotalAmountLC değerlerini 0 ile doldurma
full_customer_month_TotalAmountLC["TotalAmountLC"].fillna(0, inplace=True)

# Her bir müşterinin her bir aydaki TotalAmountLC'sini yazdır
#for customer_id, month_TotalAmountLC in full_customer_month_TotalAmountLC.groupby("Customer_id"):
    #print("Müşteri ID:", customer_id)
    #for index, row in month_TotalAmountLC.iterrows():
        #print("Aylık TotalAmountLC ({}) : {:.2f} TL".format(row["trx_month"], row["TotalAmountLC"]))


In [3]:
# TotalAmountLC'ye göre sırala
sorted_TotalAmountLC = df_selected_period.groupby("Customer_id")["TotalAmountLC"].sum().sort_values()

# Müşteri sayısını ve TotalAmountLC'yi al
customers = range(1, len(sorted_TotalAmountLC) + 1)
monthly_TotalAmountLC_values = sorted_TotalAmountLC.values


# Grafik çiz
#plt.figure(figsize=(10, 6))
#plt.plot(customers, monthly_TotalAmountLC_values, marker='o', color='blue')
#plt.title('Müşteri Sayısı ve Yuvarlanmış total TotalAmountLC (En Yüksek 20 total TotalAmountLC\'ye Sahip Olanlar Çıkarıldı)')
#plt.xlabel('Müşteri Sayısı')
#plt.ylabel('Yuvarlanmış total TotalAmountLC')
#plt.grid(True)
#plt.show()

In [4]:
# En yüksek TotalAmountLC'ye sahip olan 20 müşteriyi bul
top_20_customers = df_selected_period.groupby("Customer_id")["TotalAmountLC"].sum().nlargest(20).index

# En yüksek TotalAmountLC'ye sahip olan 20 müşteriyi veriden çıkar
df_selected_period = df_selected_period[~df_selected_period["Customer_id"].isin(top_20_customers)]

# Müşterilerin TotalAmountLC'sini yuvarla
df_selected_period["TotalAmountLC"] = df_selected_period["TotalAmountLC"].apply(lambda x: round(x, -2))

# TotalAmountLC'ye göre sırala
sorted_TotalAmountLC = df_selected_period.groupby("Customer_id")["TotalAmountLC"].sum().sort_values()

# Müşteri sayısını ve TotalAmountLC'yi al
customers = range(1, len(sorted_TotalAmountLC) + 1)
monthly_TotalAmountLC_values = sorted_TotalAmountLC.values


# Grafik çiz
#plt.figure(figsize=(10, 6))
#plt.plot(customers, monthly_TotalAmountLC_values, marker='o', color='blue')
#plt.title('Müşteri Sayısı ve Yuvarlanmış total TotalAmountLC (En Yüksek 20 total TotalAmountLC\'ye Sahip Olanlar Çıkarıldı)')
#plt.xlabel('Müşteri Sayısı')
#plt.ylabel('Yuvarlanmış total TotalAmountLC')
#plt.grid(True)
#plt.show()

In [5]:
import numpy as np

# TotalAmountLC değerlerini gruplara ayırma
bins = np.arange(0, 8996.00, 500)  # 10000 TL aralıklarla gruplama
TotalAmountLC_groups = pd.cut(sorted_TotalAmountLC, bins=bins)

# Her bir TotalAmountLC grubundaki müşteri sayısını hesaplama
customers_per_group = TotalAmountLC_groups.value_counts().sort_index()

# Grafik çiz
#plt.figure(figsize=(10, 6))
#plt.bar(np.arange(len(customers_per_group)), customers_per_group, color='blue', align='center')
#plt.title('TotalAmountLC Aralıklarına Göre Müşteri Sayısı')
#plt.xlabel('TotalAmountLC Aralıkları (10.000 TL Aralıklarla)')
#plt.ylabel('Müşteri Sayısı')
#plt.xticks(np.arange(len(customers_per_group)), customers_per_group.index, rotation=45)

# Müşteri sayılarını gösterme
#for i, val in enumerate(customers_per_group):
    #plt.text(i, val + 10, str(val), ha='center', va='bottom')

#plt.grid(True)
#plt.show()


In [6]:
# Serideki veri noktaları arasındaki eğimleri hesapla
slopes = []
for i in range(len(sorted_eight_months_TotalAmountLC) - 1):
    slope = (sorted_eight_months_TotalAmountLC.iloc[i + 1] - sorted_eight_months_TotalAmountLC.iloc[i])
    slopes.append(slope)

#print(slopes)


In [7]:
# Belirli bir müşterinin her aydaki toplam TotalAmountLC'sini bulma
customer_id = 1108

customer_monthly_TotalAmountLC = full_customer_month_TotalAmountLC[full_customer_month_TotalAmountLC["Customer_id"] == customer_id]

# Her bir ay için müşterinin TotalAmountLC değerini yazdırma
print("Müşteri ID:", customer_id)
for index, row in customer_monthly_TotalAmountLC.iterrows():
    print("Aylık TotalAmountLC ({}) : {:.2f} TL".format(row["trx_month"], row["TotalAmountLC"]))


# Çizim için verileri ayırma
months = customer_monthly_TotalAmountLC["trx_month"]
TotalAmountLC_values = customer_monthly_TotalAmountLC["TotalAmountLC"]

# Ayları tarih-saat formatına dönüştürme

months = months.dt.to_timestamp()

# Çizgi grafiği oluşturma
#plt.figure(figsize=(10, 6))
#plt.plot(months, TotalAmountLC_values, marker='o', color='blue')
#plt.title('Customer Expenditure Trend'.format(customer_id))
#plt.xlabel('Month')
#plt.ylabel('Amount of expenditure')
#plt.grid(True)
#plt.show()
#print(customer_monthly_TotalAmountLC)


Müşteri ID: 1108
Aylık TotalAmountLC (2023-07) : 479.90 TL
Aylık TotalAmountLC (2023-08) : 504.84 TL
Aylık TotalAmountLC (2023-09) : 551.69 TL
Aylık TotalAmountLC (2023-10) : 610.90 TL
Aylık TotalAmountLC (2023-11) : 591.90 TL
Aylık TotalAmountLC (2023-12) : 645.90 TL
Aylık TotalAmountLC (2024-01) : 612.90 TL
Aylık TotalAmountLC (2024-02) : 603.90 TL


### Kendin oluştur grafik

In [8]:
import matplotlib.pyplot as plt

# Müşterinin harcama verileri
harcama_verileri = [10000, 11000,  10000,5000, 10700, 10500, 11000, 11500]

# Ayları temsil eden bir dizi oluşturma (örneğin, Ocak'tan Ağustos'a kadar)
month_random = range(1, len(harcama_verileri) + 1)

# Çizgi grafiği oluşturma
#plt.figure(figsize=(10, 6))
#plt.plot(month_random, harcama_verileri, marker='o', color='blue')

#plt.xlabel('Month')
#plt.ylabel('Amount of Expenditure')
#plt.grid(True)
#plt.show()


#### Outlier çıkarılmış veri

In [9]:
mean_TotalAmountLC = customer_monthly_TotalAmountLC["TotalAmountLC"].mean()
std_TotalAmountLC = customer_monthly_TotalAmountLC["TotalAmountLC"].std()

# Aykırı değerlerin eşiklerini belirle (genellikle ±2 veya ±3 standart sapma)
threshold = 2  # veya 3, bu sizin seçiminize bağlı olarak değiştirilebilir

# Aykırı değerleri filtrele
outliers = customer_monthly_TotalAmountLC[
    (customer_monthly_TotalAmountLC["TotalAmountLC"] < mean_TotalAmountLC - threshold * std_TotalAmountLC) |
    (customer_monthly_TotalAmountLC["TotalAmountLC"] > mean_TotalAmountLC + threshold * std_TotalAmountLC)
]

# Aykırı olmayan verilerin ortalaması
non_outlier_data = customer_monthly_TotalAmountLC[~customer_monthly_TotalAmountLC.index.isin(outliers.index)]
non_outlier_mean = non_outlier_data["TotalAmountLC"].mean()

# Aykırı değerleri dışındaki verilerin ortalaması ile değiştir
customer_monthly_TotalAmountLC.loc[outliers.index, "TotalAmountLC"] = non_outlier_mean

# Her bir ay için müşterinin TotalAmountLC değerini yazdırma (filtrelenmemiş ve aykırı değerleri dışındaki verilerin ortalaması ile değiştirilmiş)
print("Müşteri ID:", customer_id)
for index, row in customer_monthly_TotalAmountLC.iterrows():
    print("Aylık TotalAmountLC ({}) : {:.2f} TL".format(row["trx_month"], row["TotalAmountLC"]))

# Çizim için verileri ayırma (filtrelenmemiş ve aykırı değerleri dışındaki verilerin ortalaması ile değiştirilmiş)
months_filtered = customer_monthly_TotalAmountLC["trx_month"]
TotalAmountLC_values_filtered = customer_monthly_TotalAmountLC["TotalAmountLC"]
months_filtered = months_filtered.dt.to_timestamp()

# Çizgi grafiği oluşturma (filtrelenmemiş ve aykırı değerleri dışındaki verilerin ortalaması ile değiştirilmiş)
#plt.figure(figsize=(10, 6))
#plt.plot(months_filtered, TotalAmountLC_values_filtered, marker='o', color='green')
#plt.title('The Expenditure Trend of the Customer {} After Replacing Outliers with Non-Outlier Mean'.format(customer_id))
#plt.xlabel('Month')
#plt.ylabel('Amount of Expenditure')
#plt.grid(True)
#plt.show()


Müşteri ID: 1108
Aylık TotalAmountLC (2023-07) : 479.90 TL
Aylık TotalAmountLC (2023-08) : 504.84 TL
Aylık TotalAmountLC (2023-09) : 551.69 TL
Aylık TotalAmountLC (2023-10) : 610.90 TL
Aylık TotalAmountLC (2023-11) : 591.90 TL
Aylık TotalAmountLC (2023-12) : 645.90 TL
Aylık TotalAmountLC (2024-01) : 612.90 TL
Aylık TotalAmountLC (2024-02) : 603.90 TL


#### Tüm müşterilerden Outlier'ları Çıkartalım

In [10]:
import pandas as pd
import numpy as np

# Boş bir liste oluştur
outlierRemoved_data = []

# Tüm müşteriler için aykırı değerleri ortalamayla değiştirme işlemi
for customer_id, group in full_customer_month_TotalAmountLC.groupby('Customer_id'):
    # Müşteri grubundaki tüm değerlerin ortalamasını hesapla
    mean_TotalAmountLC = group["TotalAmountLC"].mean()
    
    # Müşteri grubundaki aykırı değerleri bul
    mean_TotalAmountLC = group["TotalAmountLC"].mean()
    std_TotalAmountLC = group["TotalAmountLC"].std()
   
    outliers = group[(group["TotalAmountLC"] < mean_TotalAmountLC - threshold * std_TotalAmountLC) |
                     (group["TotalAmountLC"] > mean_TotalAmountLC + threshold * std_TotalAmountLC)]
    
    # Aykırı değerleri ortalama ile değiştir
    group.loc[outliers.index, "TotalAmountLC"] = mean_TotalAmountLC
    
    # İşlenmiş veriyi listeye ekle
    outlierRemoved_data.append(group)

# Listeyi birleştirerek işlenmiş veriyi bir DataFrame'e dönüştür
outlierRemoved_customer_monthly_TotalAmountLC = pd.concat(outlierRemoved_data)

print(outlierRemoved_customer_monthly_TotalAmountLC)


     Customer_id trx_month  TotalAmountLC
0           1108   2023-07      479.90000
1           1108   2023-08      504.84000
2           1108   2023-09      551.69000
3           1108   2023-10      610.90000
4           1108   2023-11      591.90000
..           ...       ...            ...
515     99178933   2023-10        0.00000
516     99178933   2023-11        0.00000
517     99178933   2023-12        0.00000
518     99178933   2024-01      538.68875
519     99178933   2024-02        0.00000

[1560 rows x 3 columns]


#### Ortalama Negative Slope Hesaplama

In [11]:
# Her bir müşteri için slope hesaplama fonksiyonu
def calculate_slope(TotalAmountLC_values):
    slopes_percent = []
    for i in range(1, len(TotalAmountLC_values)):
        slope_percent = ((TotalAmountLC_values[i] - TotalAmountLC_values[i-1]) / TotalAmountLC_values[i-1])  if TotalAmountLC_values[i-1] != 0 else 0
        slopes_percent.append(slope_percent)
    return slopes_percent

# Her bir müşteri ve her ay için slope hesaplama
customer_monthly_slopes_percent = {}

for customer_id, month_TotalAmountLC in outlierRemoved_customer_monthly_TotalAmountLC.groupby("Customer_id"):
    TotalAmountLC_values_outlierRemoved = month_TotalAmountLC["TotalAmountLC"].values
    slopes_percent = calculate_slope(TotalAmountLC_values_outlierRemoved)
    for month, slope_percent in zip(month_TotalAmountLC["trx_month"], slopes_percent):
        key = (customer_id, month)
        customer_monthly_slopes_percent[key] = slope_percent

# Sözlüğü müşteri kimliğine göre sıralama
sorted_customer_monthly_slopes_percent = sorted(customer_monthly_slopes_percent.items(), key=lambda x: x[0][0])

# Elde edilen veri çerçevesini yazdırma
for key, slope_percent in sorted_customer_monthly_slopes_percent:
    customer_id, month = key
    print("Customer ID:", customer_id, "Month:", month+1, "Slope (%):", slope_percent)

negative_slopes = [slope_percent for slope_percent in customer_monthly_slopes_percent.values() if slope_percent < 0]

average_negative_slope = np.mean(negative_slopes)

print("Negatif eğimlere sahip müşterilerin ortalama eğimi: {:.2f}%".format(average_negative_slope))
slope_2023_07 = customer_monthly_slopes_percent.get((customer_id, "2023-07"), None)
print(slope_2023_07)


Customer ID: 1108 Month: 2023-08 Slope (%): 0.05196916024171702
Customer ID: 1108 Month: 2023-09 Slope (%): 0.09280167974011584
Customer ID: 1108 Month: 2023-10 Slope (%): 0.10732476571987877
Customer ID: 1108 Month: 2023-11 Slope (%): -0.03110165329841218
Customer ID: 1108 Month: 2023-12 Slope (%): 0.0912316269640142
Customer ID: 1108 Month: 2024-01 Slope (%): -0.05109150023223409
Customer ID: 1108 Month: 2024-02 Slope (%): -0.014684287812041117
Customer ID: 1651 Month: 2023-08 Slope (%): 0.340927360733405
Customer ID: 1651 Month: 2023-09 Slope (%): -0.009183673469387756
Customer ID: 1651 Month: 2023-10 Slope (%): -0.28341915550978375
Customer ID: 1651 Month: 2023-11 Slope (%): -0.06582351250359293
Customer ID: 1651 Month: 2023-12 Slope (%): 0.13599999999999995
Customer ID: 1651 Month: 2024-01 Slope (%): 0.1572318526543879
Customer ID: 1651 Month: 2024-02 Slope (%): 0.11995318899941486
Customer ID: 1653 Month: 2023-08 Slope (%): 0.49972027073256947
Customer ID: 1653 Month: 2023-09 Slo

In [12]:
import pandas as pd

# Sözlükten DataFrame'e dönüştürme
slope_df = pd.DataFrame(columns=["Customer_id", "trxdate", "Slope"])

# Sözlükten DataFrame'e veri aktarma
for (customer_id, month), slope_percent in customer_monthly_slopes_percent.items():
    slope_df = pd.concat([slope_df, pd.DataFrame({"Customer_id": [customer_id], "trxdate": [month + 1], "Slope": [slope_percent]})], ignore_index=True)



# trxdate sütununu datetime türüne dönüştürme
slope_df["trxdate"] = slope_df["trxdate"].dt.to_timestamp()

# trxdate sütununu istenen formatta yeniden biçimlendirme
slope_df["trxdate"] = slope_df["trxdate"].dt.strftime('%Y-%m')


# Elde edilen DataFrame'i gösterme
print(slope_df.head())
slope_df.to_excel("slope.xlsx", index=False)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_3128\4143221562.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  slope_df = pd.concat([slope_df, pd.DataFrame({"Customer_id": [customer_id], "trxdate": [month + 1], "Slope": [slope_percent]})], ignore_index=True)


  Customer_id  trxdate     Slope
0        1108  2023-08  0.051969
1        1108  2023-09  0.092802
2        1108  2023-10  0.107325
3        1108  2023-11 -0.031102
4        1108  2023-12  0.091232


In [13]:
import pandas as pd

# Hareketli ortalama hesaplama ve NaN değerleri 0 ile değiştirme
MA_df = pd.DataFrame(columns=["Customer_id", "trxdate", "Moving_Average"])

for Customer_id, group in outlierRemoved_customer_monthly_TotalAmountLC.groupby("Customer_id"):
    TotalAmountLC_values = group["TotalAmountLC"].values
    slopes_percent = calculate_slope(TotalAmountLC_values)
    for i, (month, TotalAmountLC, slope_percent) in enumerate(zip(group["trx_month"]+1, TotalAmountLC_values, slopes_percent)):
        moving_average = group["TotalAmountLC"].iloc[:i].mean()
        moving_average = 0 if pd.isnull(moving_average) else moving_average
        MA_df = pd.concat([MA_df, pd.DataFrame({"Customer_id": [Customer_id], "trxdate": [month], "Moving_Average": [moving_average]})], ignore_index=True)
print(MA_df.head())
MA_df.to_excel("MA.xlsx")


C:\Users\ASUS\AppData\Local\Temp\ipykernel_3128\2109514908.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  MA_df = pd.concat([MA_df, pd.DataFrame({"Customer_id": [Customer_id], "trxdate": [month], "Moving_Average": [moving_average]})], ignore_index=True)


  Customer_id  trxdate Moving_Average
0        1108  2023-08              0
1        1108  2023-09          479.9
2        1108  2023-10         492.37
3        1108  2023-11     512.143333
4        1108  2023-12       536.8325


In [14]:
#import pandas as pd

#churn_data = []

# Tüm müşteriler için dön
#for customer_id, group in outlierRemoved_customer_monthly_TotalAmountLC.groupby("Customer_id"):
    # Müşterinin toplam harcamalarını ve eğimlerini al
    #TotalAmountLC_values = group["TotalAmountLC"].values
    #slopes_percent = calculate_slope(TotalAmountLC_values)  # slopes_percent değerini hesaplamayı unutmamalısınız

    # "2023-07" tarihine sahip satırların churn değerlerini 0 olarak ekleme
    #churn_data.append((customer_id, "2023-07", 0))

    # Diğer aylar için churn değerlerini hesaplayıp ekleme
    #for i, (month, TotalAmountLC, slope_percent) in enumerate(zip(group["trx_month"]+1, TotalAmountLC_values, slopes_percent)):
        # "2023-07" ayını zaten eklediğimiz için tekrar eklemiyoruz
        #if month != "2023-07":
            # Müşterinin eğimi ortalama negatif eğimden küçük ve harcaması aya kadar olan harcamalarının ortalamasından küçükse churn olarak işaretle
            #if slope_percent < average_negative_slope and TotalAmountLC < group["TotalAmountLC"].iloc[:i].mean():
                #churn_data.append((customer_id, month, 1))  # Churn
            #elif TotalAmountLC == 0:  # Eğer müşteri o ay harcama yapmadıysa churn olarak işaretle
                #churn_data.append((customer_id, month, 1))  # Churn
            #else:
                #churn_data.append((customer_id, month, 0))  # Churn değil

# Churn verisini DataFrame'e dönüştür
#churn_df = pd.DataFrame(churn_data, columns=["Customer_id", "trxdate", "Churn"])
#churn_df.to_excel("churn_analysis.xlsx", index=False)

# Churn olan müşteri sayısını bul
#churn_count = churn_df["Churn"].sum()

# Toplam ay sayısını bul
#total_months = len(churn_df)

# Churn oranını hesapla
#churn_rate = churn_count / total_months

#print(churn_df)
#print("Toplam Ay Sayısı:", total_months)
#print("Churn Olma Sayısı:", churn_count)
#print("Churn Gerçekleşme Oranı: {:.2%}".format(churn_rate))
